In [48]:
import os
import pandas as pd
import numpy as np

from scipy import stats
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import LocalOutlierFactor

import matplotlib.pyplot as plt
import seaborn as sns


In [49]:
from sklearn.metrics import confusion_matrix,classification_report

In [50]:
data = pd.read_csv('1_Iris_virginica.csv')

In [51]:
data.head(10)

,sepallength,sepalwidth,petallength,petalwidth,label
0,5.1,3.5,1.4,0.2,outlier
1,4.9,3.0,1.4,0.2,outlier
2,4.7,3.2,1.3,0.2,outlier
3,4.6,3.1,1.5,0.2,outlier
4,5.0,3.6,1.4,0.2,outlier
5,5.4,3.9,1.7,0.4,outlier
6,4.6,3.4,1.4,0.3,outlier
7,5.0,3.4,1.5,0.2,outlier
8,4.4,2.9,1.4,0.2,outlier
9,4.9,3.1,1.5,0.1,outlier


In [52]:
from sklearn.model_selection import train_test_split

In [53]:
train_data, test_data = train_test_split(data,test_size=0.2)

In [54]:
Train_Input = train_data.drop(['label'],axis=1)
Train_output = train_data['label']
Train_Input_x1 = Train_Input.iloc[:,0]

In [55]:
Train_Input.shape

(120, 4)

In [56]:
Train_output.shape

(120,)

In [57]:
Train_Input_x1.shape

(120,)

In [58]:
Train_Input_target = train_data.loc[train_data['label']=='target'].drop(['label'],axis=1)

In [59]:
Train_Input_target.shape

(39, 4)

In [60]:
Train_Input_outlier =train_data.loc[train_data['label']=='outlier'].drop(['label'],axis=1)

In [61]:
Train_Input_outlier.shape

(81, 4)

In [62]:
Train_Input_target_x1 = Train_Input.iloc[:,0]

In [63]:
Train_Input_outlier_x1 = Train_Input_outlier.iloc[:,0]

In [64]:
est_mean = np.mean(Train_Input_target_x1)
est_std = np.std(Train_Input_target_x1)

In [65]:
train_anomaly_score = stats.norm.pdf(Train_Input_x1,loc=est_mean,scale=est_std) 

In [66]:
score = stats.norm.pdf(Train_Input_outlier_x1,loc=est_mean,scale=est_std)

In [67]:
train_anomaly_score

array([0.37808497, 0.52293271, 0.39302094, 0.21243921, 0.49369795,
       0.41788551, 0.29356797, 0.51425664, 0.5076579 , 0.2266097 ,
       0.02072575, 0.35164452, 0.35164452, 0.41788551, 0.26701476,
       0.25193925, 0.41788551, 0.25193925, 0.30913182, 0.52926375,
       0.30913182, 0.37808497, 0.26701476, 0.39302094, 0.41788551,
       0.2266097 , 0.51425664, 0.52632038, 0.2266097 , 0.51425664,
       0.37808497, 0.5076579 , 0.15481749, 0.48422608, 0.25193925,
       0.49369795, 0.49369795, 0.41788551, 0.5076579 , 0.46568733,
       0.25193925, 0.09857535, 0.41788551, 0.33610362, 0.17600509,
       0.35164452, 0.33610362, 0.41788551, 0.49369795, 0.17600509,
       0.30913182, 0.51425664, 0.52926375, 0.52632038, 0.45381297,
       0.2266097 , 0.09857535, 0.15481749, 0.46568733, 0.51425664,
       0.46568733, 0.37808497, 0.30913182, 0.52293271, 0.35164452,
       0.41788551, 0.30913182, 0.29356797, 0.48422608, 0.30913182,
       0.51425664, 0.5076579 , 0.41788551, 0.52632038, 0.21243

In [68]:
threshold = 0.05

In [69]:
train_anomaly_score < 0.05

array([False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [70]:
train_anomaly_score = stats.norm.pdf(Train_Input_x1,loc=est_mean,scale=est_std)

In [71]:
from sklearn.covariance import EmpiricalCovariance

In [72]:
cov_est_model =EmpiricalCovariance()

In [73]:
cov_est_model.fit(Train_Input_target)

EmpiricalCovariance()

In [74]:
est_multi_mean = cov_est_model.location_
est_multi_cov = cov_est_model.covariance_

In [75]:
train_anomaly_score_multi = stats.multivariate_normal.pdf(Train_Input,mean = est_multi_mean,cov =est_multi_cov)

In [76]:
est_mean = np.mean(Train_Input)
est_std = np.std(Train_Input)

In [77]:
print(est_std)

sepallength    0.753545
sepalwidth     0.431663
petallength    1.733281
petalwidth     0.773978
dtype: float64


In [78]:
pred = []

for i in range(train_anomaly_score.shape[0]):
        
    if train_anomaly_score_multi[i] < threshold:
         pred.append('outlier')
            
    else:
        pred.append('target')

In [79]:
Train_Input_x1

111    6.4
61     5.9
28     5.2
76     6.8
80     5.5
      ... 
65     6.7
141    6.9
79     5.7
125    7.2
1      4.9
Name: sepallength, Length: 120, dtype: float64

In [80]:
df = pd.DataFrame(pred)

In [81]:
df.head()

,0
0,target
1,outlier
2,outlier
3,outlier
4,outlier


In [82]:
df[0].value_counts()

outlier    81
target     39
Name: 0, dtype: int64

In [83]:
len(train_anomaly_score)

120

In [84]:
train_anomaly_score.shape

(120,)

In [85]:
confusion_matrix(Train_output,pred,labels = ['target','outlier'])

array([[36,  3],
       [ 3, 78]], dtype=int64)

In [86]:
classification_report(Train_output,pred,labels = ['target','outlier'], output_dict = True)

{'target': {'precision': 0.9230769230769231,
  'recall': 0.9230769230769231,
  'f1-score': 0.9230769230769231,
  'support': 39},
 'outlier': {'precision': 0.9629629629629629,
  'recall': 0.9629629629629629,
  'f1-score': 0.9629629629629629,
  'support': 81},
 'accuracy': 0.95,
 'macro avg': {'precision': 0.9430199430199431,
  'recall': 0.9430199430199431,
  'f1-score': 0.9430199430199431,
  'support': 120},
 'weighted avg': {'precision': 0.95,
  'recall': 0.95,
  'f1-score': 0.95,
  'support': 120}}

In [87]:
LOF = LocalOutlierFactor()

In [88]:
LOF.fit(Train_Input)

LocalOutlierFactor()

In [89]:
np.unique(Train_output,return_counts=True)

(array(['outlier', 'target'], dtype=object), array([81, 39], dtype=int64))

In [90]:
nof = LOF.negative_outlier_factor_[-1 * LOF.negative_outlier_factor_.argsort()]

In [91]:
pred = 120 * ['target']

In [92]:
anomal_ind =LOF.negative_outlier_factor_.argsort()[:83]

In [93]:
for i in anomal_ind:
    pred[i] = 'outlier'

In [94]:
confusion_matrix(Train_output,pred,labels=['target'])

array([[12]], dtype=int64)